In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import json
import MDAnalysis as mda
import nglview as nv

In [ ]:
PATH = 'data/SIU'

pockets = json.load(open(os.path.join(PATH, "target_info.json")))
len(pockets.keys())

In [ ]:
pdb_id = "1zab"
u = mda.Universe(os.path.join(PATH, "pdb_files", f"{pdb_id}.pdb"))
view = nv.show_mdanalysis(u)
view.add_representation("surface", colorScheme="hydrophobicity")
view.shape.add_sphere(pockets[pdb_id]['center'], [0,1,1], np.min(pockets[pdb_id]['size']), "pocket")
view

In [ ]:
u = mda.Universe(os.path.join(PATH, "pdb_files", f"{pdb_id}.pdb"))
view = nv.show_mdanalysis(u)
view.add_representation("surface", colorScheme="hydrophobicity")
# view.shape.add_sphere(pockets[pdb_id]['center'], [0,1,1], np.min(pockets[pdb_id]['size']), "pocket")
view